# YAHOO電影爬蟲練習
## 練習爬取電影放映資訊。必須逐步獲取電影的代號、放映地區、放映日期後，再送出查詢給伺服器。

### 先搜尋全部的電影代號(ID)資訊

In [105]:
import requests
import re
from bs4 import BeautifulSoup  # 查看目前上映那些電影，並擷取出其ID資訊

req = requests.get('https://movies.yahoo.com.tw/movie_intheaters.html')
req.encoding = 'utf-8'

response = BeautifulSoup(req.text)
select = response.find('select', id='sbox_mid')
id_dict = {}
for option in select.find_all('option'):
    if option['value'] != '':
        id_dict[option['data-name']] = option['value']
print(id_dict)

{'復仇母親': '10392', '風暴過後': '10387', '南方車站的聚會': '10380', '祭旗': '10373', '唯你是愛': '10370', '屍控療法': '10367', '黑熊來了': '10357', '唐人街探案2': '10353', '黑計畫': '10351', 'HUMAN LOST人間失格': '10348', '假面騎士劇場版ZI-O Over Quartzer': '10331', '巴黎小情聖': '10329', '每個月來訪一次的月經醬': '10324', '8級警戒': '10323', '為你存在的每一天': '10297', '黑色聖誕節': '10291', '熱氣球飛行家': '10257', '大說謊家': '10122', '特約經紀公司': '10364', '一屍到底番外篇：好萊塢大作戰！': '10352', '花生醬獵鷹的願望': '10340', '女孩要革命': '10339', '我的靈魂是愛做的': '10336', '絕世情歌': '10328', '人間失格：太宰治與他的3個女人': '10287', '抓狂博物館': '10279', '呆萌特務': '10264', '去年聖誕節': '10256', '野蠻遊戲：全面晉級': '10134', '夕霧花園': '10327', '懸案密碼前傳：瓶中信': '10254', '布魯克林孤兒': '10224', '暴走曼哈頓': '10044', '鋒迴路轉': '10258', '賽道狂人': '10097', '麂皮：永不滿足': '10358', '82年生的金智英': '10298', '冰雪奇緣2': '9597', 'Hello World': '10302', '大餓': '10296', '我的情敵是冰箱': '10286', '萬萬沒想到': '10281', '為美好的世界獻上祝福！紅傳說': '10226', '山田孝之的痛苦與榮耀': '10301', '決戰中途島': '10130', '安眠醫生': '10098', '地久天長': '9621', '魔鬼終結者：黑暗宿命': '10067', '陽光普照': '10240', '狙擊獵殺：救援行動': '10284', '雙子殺手': '

### 指定你有興趣的電影其ID，然後查詢其放映地區資訊。

In [106]:
# 參考前一個步驟中擷取到的ID資訊，並指定ID
movie_id = id_dict['去年聖誕節']

In [107]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id': movie_id}
headers = {
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb'
}
response = requests.get(url, params=payload, headers=headers)
# print(response.json())
area_dict = {}
for r in response.json():
    area_dict[r['title']] = r['area_id']
#     print('地區: %s, 代號: %d' % (r['title'], r['area_id']))
print(area_dict)

{'台北市': 28, '新北市': 8, '桃園': 16, '新竹': 20, '苗栗': 15, '台中': 2, '彰化': 22, '雲林': 19, '嘉義': 21, '台南': 10, '高雄': 17, '宜蘭': 11, '花蓮': 12, '台東': 9, '金門': 24, '澎湖': 23}


### 指定你想要觀看的放映地區，查詢有上映電影的場次日期

In [108]:
# 指定放映地區
area_id = area_dict['桃園']

In [109]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':movie_id, 'area_id':str(area_id)}
req = requests.get(url, params=payload)
req.encoding = 'utf-8'

# 列印播放日期
response = BeautifulSoup(req.text)
date_list = []
for d in response.find_all('input', attrs={'name':'date', 'type':'radio'}):
    try:
        date_list.append(d['value'])
        print('播放日期: %s' % d['value'])
    except:
        continue


播放日期: 2019-12-14
播放日期: 2019-12-15
播放日期: 2019-12-16
播放日期: 2019-12-17


### 最後指定觀看的日期，查詢並列印出放映的電影院、放映類型(數位、3D、IMAX 3D...)、放映時間等資訊。

In [110]:
# 選定要觀看的日期
date = date_list[0]

In [137]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':movie_id,
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

req = requests.get(url, params=payload)
req.encoding = 'utf-8'
# print(bytes(req.text.encode()).decode('unicode-escape'))
cinema_list = []
response = BeautifulSoup(req.json()['view'])
for ul in response.find_all('ul', id=re.compile('theater_id_\d+')):
    cinema_list.append({'name':ul['data-theater_name'], 'schedules':[]})
    inps = BeautifulSoup(str(ul)).find_all('input', attrs={'name':'schedule_list'})
    for inp in inps:
        cinema_list[-1]['schedules'].append({'類型':inp['data-movie_type'], '時間':inp['value']})

In [138]:
'''
  試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表
'''
for c in cinema_list:
    print('電影院：%s' % c['name'])
    for sch in c['schedules']:
        print('  %s %s' % (sch['類型'], sch['時間']))

電影院：SBC星橋國際影城
  數位 2019-12-14 10:00:00
  數位 2019-12-14 14:30:00
  數位 2019-12-14 19:05:00
  數位 2019-12-14 21:25:00
電影院：威尼斯影城
  數位 2019-12-14 10:00:00
  數位 2019-12-14 13:10:00
  數位 2019-12-14 15:20:00
  數位 2019-12-14 18:30:00
  數位 2019-12-14 20:40:00
  數位 2019-12-14 23:50:00
  數位 2019-12-15 02:00:00
電影院：桃園in89統領影城
  數位 2019-12-14 20:30:00
電影院：國賓影城(桃園八德廣豐新天地)
  數位 2019-12-14 13:15:00
  數位 2019-12-14 17:55:00
  數位 2019-12-14 22:40:00
電影院：桃園統領威秀影城
  數位 2019-12-14 19:00:00
  數位 2019-12-14 23:00:00
